# Part 3a - Data Transformations

- **Objective**: Learn about integrating data transformations and interaction into a chart definition.
- **Key Topics**:
  - Binning and aggregation
  - Basic Interactive features (tooltips and pan/zoom)
  - Selections parameters
  - Conditional encodings and filtering

In the previous section we learned how to use marks and visual encodings to represent individual data records. In this section, we will explore methods for *transforming* data, including the use of aggregates to summarize multiple records. Data transformation is an integral part of visualization: choosing the variables to show and their level of detail is just as important as choosing appropriate visual encodings. After all, it doesn't matter how well chosen your visual encodings are if you are showing the wrong information!

As we work through this module, it may be helpful to open the [Altair Data Transformations](https://altair-viz.github.io/user_guide/transform/index.html) documentation in another tab. It will be a useful resource if at any point you'd like more details or want to see what other transformations are available.

## Credits
This section is largely based on the [Visualization Curriculum](https://idl.uw.edu/visualization-curriculum/intro.html) developed at the University of Washington by Jeffrey Heer, Dominik Moritz, Jake VanderPlas, and Brock Craft. Additional thanks to Christopher Davis for his work to create the base materials for this notebook.


## Imports
Perform the package imports that will be needed for this section of the tutorial

In [ ]:
import pandas as pd
import altair as alt
from vega_datasets import data
import vegafusion

print("The installed Vega-Altair version is " + alt.__version__)

## The Movies Dataset
We will be working with a table of data about motion pictures, taken from the vega-datasets collection which is provided by the `vega_datasets` Python package. The dataset includes variables such as the film name, director, genre, release date, ratings, and gross revenues. However, *be careful when working with this data*: the films are from unevenly sampled years, using data combined from multiple sources. If you dig in you will find issues with missing values and even some subtle errors! Nevertheless, the data should prove interesting to explore…


Let's load the dataset into a pandas DataFrame using the `vega_datasets` package so that we can inspect its contents.


In [ ]:
movies = data.movies()

How many rows (records) and columns (fields) are in the movies dataset?

In [ ]:
movies.shape

Now let’s peek at the first 5 rows of the table to get a sense of the fields and data types…

In [ ]:
movies.head(5)

## Histograms
We’ll start our transformation tour by *binning* data into discrete groups and *counting* records to summarize those groups. The resulting plots are known as [*histograms*](https://en.wikipedia.org/wiki/Histogram).

Let’s first look at the unaggregated data: a scatter plot showing movie ratings from Rotten Tomatoes versus ratings from IMDB users. We'll provide data to Altair by passing the `movies` DataFrame to the `Chart` constructor.  We can then encode the Rotten Tomatoes and IMDB ratings fields using the `x` and `y` channels:

In [ ]:
alt.Chart(movies).mark_circle().encode(
    alt.X('Rotten_Tomatoes_Rating'),
    alt.Y('IMDB_Rating')
)

To summarize this dataset, we can bin a data field to group numeric values into discrete groups. Here we bin along the x-axis by adding `.bin()` to the `x` encoding channel. The result is a set of ten bins of equal step size, each corresponding to a span of ten ratings points.


In [ ]:
alt.Chart(movies).mark_circle().encode(
    alt.X('Rotten_Tomatoes_Rating').bin(),
    alt.Y('IMDB_Rating:Q')
)

Adding `.bin()` uses default binning settings, but we can exercise more control if desired. Let's instead set the maximum bin count (`maxbins`) to 20, which has the effect of doubling the number of bins. Now each bin corresponds to a span of five ratings points.

In [ ]:
alt.Chart(movies).mark_circle().encode(
    alt.X('Rotten_Tomatoes_Rating').bin(maxbins=20),
    alt.Y('IMDB_Rating:Q')
)

It's important to note that binning, by itself, does not change the number of displayed points, it only discretizes the values of a dimension of the data, typically causing points to overlap with each other.

With the data binned, let's now summarize the distribution of Rotten Tomatoes ratings. We will drop the IMDB ratings for now and instead use the `y` encoding channel to show an aggregate `count` of records, so that the vertical position of each point indicates the number of movies per Rotten Tomatoes rating bin.

As the `count` aggregate counts the number of total records in each bin regardless of the field values, we do not need to include a field name in the `y` encoding.

In [ ]:
alt.Chart(movies).mark_circle().encode(
    alt.X('Rotten_Tomatoes_Rating').bin(maxbins=20),
    alt.Y('count()')
)

To arrive at a standard histogram, let's change the mark type from `circle` to `bar`:

In [ ]:
alt.Chart(movies).mark_bar().encode(
    alt.X('Rotten_Tomatoes_Rating').bin(maxbins=20),
    alt.Y('count()')
)

> **Observations:** We can now examine the distribution of ratings more clearly. we can see fewer movies on the negative end, and a bit more movies on the high end, but a generally uniform distribution overall. Rotten Tomatoes ratings are determined by taking “thumbs up” and “thumbs down” judgments from film critics and calculating the percentage of positive reviews. It appears this approach does a good job of utilizing the full range of rating values.


Similarly, we can create a histogram for IMDB ratings by changing the field in the x encoding channel:

In [ ]:
alt.Chart(movies).mark_bar().encode(
    alt.X('IMDB_Rating').bin(maxbins=20),
    alt.Y('count()')
)

> **Observation:** In contrast to the more uniform distribution we saw before, IMDB ratings exhibit a bell-shaped (though [negatively skewed](https://en.wikipedia.org/wiki/Skewness)) distribution. IMDB ratings are formed by averaging scores (ranging from 1 to 10) provided by the site's users. We can see that this form of measurement leads to a different shape than the Rotten Tomatoes ratings. We can also see that the mode of the distribution is between 6.5 and 7: people generally enjoy watching movies, potentially explaining the positive bias!

Now let's turn back to our scatter plot of Rotten Tomatoes and IMDB ratings. Here's what happens if we bin both axes of our original plot.


In [ ]:
alt.Chart(movies).mark_circle().encode(
    alt.X('Rotten_Tomatoes_Rating').bin(maxbins=20),
    alt.Y('IMDB_Rating').bin(maxbins=20)
)

Detail is lost due to *overplotting*, with many points drawn directly on top of each other.

To form a two-dimensional histogram we can add a `count` aggregate as before. As both the `x` and `y` encoding channels are already taken, we must use a different encoding channel to convey the counts. Here is the result of using circular area by adding a size encoding channel.



In [ ]:
alt.Chart(movies).mark_circle().encode(
    alt.X('Rotten_Tomatoes_Rating').bin(maxbins=20),
    alt.Y('IMDB_Rating').bin(maxbins=20),
    alt.Size('count()')
)

Alternatively, we can encode counts using the `color` channel and change the mark type to `rect`. The result is a two-dimensional histogram in the form of a [heatmap](https://en.wikipedia.org/wiki/Heat_map).

In [ ]:
alt.Chart(movies).mark_rect().encode(
    alt.X('Rotten_Tomatoes_Rating').bin(maxbins=20),
    alt.Y('IMDB_Rating').bin(maxbins=20),
    alt.Color('count()')
)

Compare the size and color-based 2D histograms above. Which encoding do you think should be preferred? Why? In which plot can you more precisely compare the magnitude of individual values? In which plot can you more accurately see the overall density of ratings?

## Aggregation
Counts are just one type of aggregate. We might also calculate summaries using measures such as the `average`, `median`, `min`, or `max`. The Altair documentation includes the [full set of available aggregation functions](https://altair-viz.github.io/user_guide/encodings/index.html#aggregation-functions).

Let's look at some more examples!

### Averages and Sorting
> **Question:** Do different genres of films receive consistently different ratings from critics?

As a first step towards answering this question, we might examine the *average* (a.k.a. the [arithmetic mean](https://en.wikipedia.org/wiki/Arithmetic_mean)) rating for each genre of movie.

Let's visualize genre along the `y` axis and plot `average` Rotten Tomatoes ratings along the `x` axis.

In [ ]:
alt.Chart(movies).mark_bar().encode(
    alt.X('average(Rotten_Tomatoes_Rating)'),
    alt.Y('Major_Genre')
)

> **Observation:** There does appear to be some interesting variation, but looking at the data as an alphabetical list is not very helpful for ranking critical reactions to the genres.

For a tidier picture, let's sort the genres in descending order of average rating. To do so, we will use the `.sort()` method on the y encoding channel, stating that we wish to sort by the bar x positions in descending order.

In [ ]:
alt.Chart(movies).mark_bar().encode(
    alt.X('average(Rotten_Tomatoes_Rating)'),
    alt.Y('Major_Genre').sort('-x')
)

> **Observation:** The sorted plot suggests that critics think highly of documentaries, musicals, westerns, and dramas, but look down upon romantic comedies and horror films… and who doesn’t love `null` movies!?

### Medians and the Inter-Quartile Range

While averages are a common way to summarize data, they can sometimes mislead. For example, very large or very small values ([*outliers*](https://en.wikipedia.org/wiki/Outlier)) might skew the average. To be safe, we can compare the genres according to the [*median*](https://en.wikipedia.org/wiki/Median) ratings as well.

The median is a point that splits the data evenly, such that half of the values are less than the median and the other half are greater. The median is less sensitive to outliers and so is referred to as a [*robust statistic*](https://en.wikipedia.org/wiki/Robust_statistics). For example, arbitrarily increasing the largest rating value will not cause the median to change.

Let’s update our plot to use a `median` aggregate and sort by those values:




In [ ]:
alt.Chart(movies).mark_bar().encode(
    alt.X('median(Rotten_Tomatoes_Rating)'),
    alt.Y('Major_Genre').sort('-x')
)

> **Observation:** We can see that some of the genres with similar averages have swapped places (films of unknown genre, or `null`, are now rated highest!), but the overall groups have stayed stable. Horror films continue to get little love from professional film critics.

It's a good idea to stay skeptical when viewing aggregate statistics. So far we've only looked at *point estimates*. We have not examined how ratings vary within a genre.

Let's visualize the variation among the ratings to add some nuance to our rankings. Here we will encode the [inter-quartile range (IQR)](https://en.wikipedia.org/wiki/Interquartile_range) for each genre. The IQR is the range in which the middle half of data values reside. A quartile contains 25% of the data values. The inter-quartile range consists of the two middle quartiles, and so contains the middle 50%.

To visualize ranges, we can use the `x` and `x2` encoding channels to indicate the starting and ending points. We use the aggregate functions `q1` (the lower quartile boundary) and `q3` (the upper quartile boundary) to provide the inter-quartile range. (In case you are wondering, *q2* would be the median.)

Rather than sort the bars by the `x` encoding, we'll use a more general form of sorting that allows us to sort by a specific aggregation `op` on a specific `field` in a specific `order`.

In [ ]:
alt.Chart(movies).mark_bar().encode(
    alt.X('q1(Rotten_Tomatoes_Rating)'),
    alt.X2('q3(Rotten_Tomatoes_Rating)'),
    alt.Y('Major_Genre').sort(
        op='median',
        field='Rotten_Tomatoes_Rating',
        order='descending'
    )
)

Here we add a `tick` mark to indicate the median for each genre.  Note that the second chart defined below automatically inherits the y-encoding (and data) from the `base` chart.

In [ ]:
base = alt.Chart(movies).mark_bar().encode(
    alt.X('q1(Rotten_Tomatoes_Rating)'),
    alt.X2('q3(Rotten_Tomatoes_Rating)'),
    alt.Y('Major_Genre').sort(
        op='median',
        field='Rotten_Tomatoes_Rating',
        order='descending'
    )
)

tick = base.mark_tick(color="red").encode(
    alt.X('median(Rotten_Tomatoes_Rating)')
)

base+tick

### Time Units
> **Question:** Do box office returns vary by season?

To get an initial answer, let’s plot the median U.S. gross revenue by month.

To make this chart, use the `timeUnit` transform to map release dates to the `month` of the year. The result is similar to binning, but using meaningful time intervals. Other valid time units include `year`, `quarter`, `date` (numeric day in month), `day` (day of the week), and `hours`, as well as compound units such as `yearmonth` or `hoursminutes`. See the Vega-Altair documentation for a [complete list of time units](https://altair-viz.github.io/user_guide/transform/timeunit.html).

In [ ]:
alt.Chart(movies).mark_area().encode(
    alt.X('month(Release_Date)'),
    alt.Y('median(US_Gross)')
)

> **Observation:** Looking at the resulting plot, median movie sales in the U.S. appear to spike around the summer blockbuster season and the end of year holiday period. Of course, people around the world (not just the U.S.) go out to the movies.

---

> **Question**: Does a similar pattern arise for worldwide gross revenue?

In [ ]:
alt.Chart(movies).mark_area().encode(
    alt.X('month(Release_Date)'),
    alt.Y('median(Worldwide_Gross)')
)

> **Observation:** Yes!

## Data Transform Architecture
By default, the entire input DataFrame is serialized to JSON and transfered from the Python Kernel to the browser, before being transformed and displayed by the Vega JavaScript library.

We can see this by creating a Histogram chart and then selecting "Open in Vega Editor" from the dropdown menu. The Vega-Lite JSON specification contains every row and column from the movies DataFrame.


In [ ]:
alt.Chart(movies).mark_bar().encode(
    alt.X('IMDB_Rating').bin(),
    alt.Y('count()')
)

For large datasets with many columns, this approach can result in very large chart specifications, which can be slow to serialize to JSON, slow to display, and lead to large notebooks. Because of these downsides, Vega-Altair enforces a default limit of 5000 rows.

We can improve the situation using the `"vegafusion"` data transformer, which uses the optional VegaFusion dependency to evaluate most Vega-Altair transforms in the Python kernel before inlining the transformed data into the resulting chart specification.

Let's repeat the process above after enabling this data transformer. Now when selecting "Open in Vega Editor, we see the transformed dataset is included in the spec. This transformed dataset includes one row per histogram bar with fields for the bar height and bin edges.

In [ ]:
alt.data_transformers.enable("vegafusion")
alt.Chart(movies).mark_bar().encode(
    alt.X('IMDB_Rating').bin(),
    alt.Y('count()')
)

> **Note:** The VegaFusion data transformer is also the recommended way to overcome Vega-Altair's default limit of 5000 rows.

## Extracting Transformed Data
Transformed data can be extracted in Python using the `.transformed_data()` chart method. This is also powered by VegaFusion, and can be very helpful for analysis as it avoids the need to repeat calcultions in pandas.

In [ ]:
chart = alt.Chart(movies).mark_bar().encode(
    alt.X('IMDB_Rating').bin(),
    alt.Y('count()')
)
chart.transformed_data()

## Advanced Transforms

The examples above all use transformations (*bin*, *timeUnit*, *aggregate*, *sort*) that are defined relative to an encoding channel. However, at times you may want to apply a chain of multiple transformations prior to visualization, or use transformations that don't integrate into encoding definitions. For such cases, Vega-Altair supports data transformations defined separately from encodings. While many transformations can be specified directly in pandas, Vega-Altair transformations can be *parameterized* such that on-chart interactions can control these transforms.

For a more thorough introduction to advanced transformations, we recommend taking a look at the [2024 SciPy tutorials part 2](https://github.com/vega/SciPy2024-Altair-Tutorial). 



### Filter
The `filter` transform creates a new table with a subset of the original data, removing rows that fail to meet a provided [*predicate*](https://en.wikipedia.org/wiki/Predicate_%28mathematical_logic%29) test. Similar to the `calculate` transform, filter predicates are expressed using the [Vega expression language](https://vega.github.io/vega/docs/expressions/).

Below we add a filter to limit our initial scatter plot of IMDB vs. Rotten Tomatoes ratings to only films in the major genre of “Romantic Comedy”.

In [ ]:
alt.Chart(movies).mark_circle().transform_filter(
    'datum.Major_Genre == "Romantic Comedy"'
).encode(
    alt.X('Rotten_Tomatoes_Rating'),
    alt.Y('IMDB_Rating')
)

> **Question:** How does the plot change if we filter to view other genres? Edit the filter expression to find out!

Now let's filter to look at films released before 1970.

**Note:** The Vega expression language provides a convenient `year` function that we can use to extract the year from a datetime column.

In [ ]:
alt.Chart(movies).mark_circle().transform_filter(
    'year(datum.Release_Date) < 1970'
).encode(
    alt.X('Rotten_Tomatoes_Rating'),
    alt.Y('IMDB_Rating')
)

> **Observation:** They seem to score unusually high! Are older films simply better, or is there a selection bias towards more highly-rated older films in this dataset?

## Part 3b - Interactivity in Vega-Altair

Vega-Altair allows you to create interactive charts that can be embedded in web pages or Jupyter notebooks (without requiring a live Python kernel). The interactivity features include tooltips, panning, zooming, selections, and more complex interactions such as linked views and coordinated highlighting. These features enable dynamic data exploration and enhance the end user experience by making visualizations more engaging and insightful.

Compared to static charts, interactive charts provide a more personalized experience, allowing end users to ask their own questions and focus on their particular areas of interest.

### Dataset

The cars dataset from vega_datasets contains information about various car models from the 1970s and 1980s. 
It includes properties like the horsepower, miles per gallon, origin.


In [ ]:
cars = data.cars()
cars.head()

### Example: Support pan, zoom, and tooltips

In [ ]:
# Create chart with tooltip
alt.Chart(cars).mark_circle(size=60).encode(
    alt.X('Horsepower'),
    alt.Y('Miles_per_Gallon').title("Miles per Gallon"),
    alt.Color('Origin'),
    tooltip=[
        'Name',
        'Origin',
        'Horsepower',
        alt.Tooltip('Miles_per_Gallon').title("Miles per Gallon")
    ]
).interactive()

### Example: Support end-user filtering

In [ ]:
options = ['Europe', 'Japan', 'USA']
labels = [option + ' ' for option in options]

input_radio = alt.binding_radio(
    options=options + [None],
    labels=labels + ['All'],
    name='Region: '
)
selection = alt.selection_point(
    fields=['Origin'],
    value=None,
    bind=input_radio,
)

alt.Chart(cars).mark_circle(size=60).encode(
    alt.X('Horsepower:Q'),
    alt.Y('Miles_per_Gallon:Q'),
    alt.Color('Origin:N').scale(domain=options),
).add_params(
    selection
).transform_filter(
    selection
)

### Example: Support multi-view highlighting

Interactive charts also enable the exploration higher dimensional relationships in the dataset through the use of selections across multi-view charts.

In [ ]:
brush = alt.selection_interval(encodings=['x'])

left_chart = alt.Chart(cars).mark_circle(size=60).encode(
    alt.X('Horsepower'),
    alt.Y('Miles_per_Gallon'),
    color=alt.condition(brush, 'Origin', alt.value('lightgray'))
).add_params(
    brush
)

right_chart = alt.Chart(cars).mark_circle(size=60).encode(
    alt.X('Acceleration'),
    alt.Y('Miles_per_Gallon'),
    color=alt.condition(brush, 'Origin', alt.value('lightgray'))
).add_params(
    brush
)

left_chart | right_chart

## Basic Interactive Features

Vega-Altair provides high-level support for two basic forms of interactivity: Tooltips and Pan/Zoom interactions.

### Tooltips

As seen already in Part 1, tooltips provide additional information when a user hovers over a data point on the chart. This feature is particularly useful for displaying detailed data without cluttering the visualization.

Vega-Altair provides a `tooltip` encoding channel that may be used to both activate tooltip support, and specify which data values should appear in the tooltips.

In [ ]:
# Create chart with tooltip
alt.Chart(cars).mark_circle(size=60).encode(
    alt.X('Horsepower'),
    alt.Y('Miles_per_Gallon'),
    alt.Color('Origin'),
    tooltip=['Name', 'Origin', 'Horsepower', 'Miles_per_Gallon']
)

Customize tooltip title with `alt.Tooltip`

In [ ]:
# Create chart with tooltip
alt.Chart(cars).mark_circle(size=60).encode(
    alt.X('Horsepower'),
    alt.Y('Miles_per_Gallon').title("Miles per Gallon"),
    alt.Color('Origin'),
    tooltip=[
        'Name',
        'Origin',
        'Horsepower',
        alt.Tooltip('Miles_per_Gallon').title("Miles per Gallon")
    ]
)

## Selection Parameters

A selection parameter represents a predicate expression (more on this below) that may be used to determine whether each row of a dataset is included in the selection or not.

Selection parameters may be bound to input widgets, or to interactions on the chart itself such as clicking on marks or clicking and dragging to create a box selection.

### What's a Predicate?
A predicate is a function or expression that evaluates to a boolean value (true or false). For example, in the context of the cars dataset we've been using, the expression `Origin == "Japan"` is a predicate that evaluates to true for cars with `Origin` of `Japan`, and false otherwise.  The exact predicate that a selection parameter represents is not always obvious, but viewing them as predicates is helpful in understanding how selections may be used.

### Conditions & Filters
Selection parameters can be used to influence the appearance of a chart in two ways: Filter and conditions.

#### Filter
The `.transform_filter()` method that we discussed in the previous section can accept a selection instance. In this case, the filter transform will remove all rows that don't satisfy the selection parameter's underlying predicate.

#### Condition
Selection parameters may also be used to control visual encoding channels using the `alt.condition()` function. A common scenario is the set the color or opacity of marks based on the selection, but the selection parameter may be used to control any encoding channel.

### Point selections
Vega-Altair supports two selection parameter types: *Point selections* and *interval selections*. Let's look at point selections first. A point selection represents a predicate of the form `{Column} == {value}`. In the case of the cars dataset, one such example would be `Origin == "Japan"`.  A point selection is constructed using the `alt.selection_point` function.

Here is a basic example of configuring a selection parameter with a fixed predicate (equivalent to `Origin == "Japan"`), and then using the selection to filter the chart's underlying data.

In [1]:
selection = alt.selection_point(
    fields=['Origin'],
    value="Japan",
)

alt.Chart(cars).transform_filter(
    selection
).mark_point().encode(
    x='Horsepower:Q',
    y='Miles_per_Gallon:Q',
    color=alt.Color('Origin:N').scale(domain=['Europe', 'Japan', 'USA']),
).add_params(
    selection
)

NameError: name 'alt' is not defined

> **Note:** Encoding channels that use a condition must be assigned to the corresponding keyword argument in the `.encode()` method call. For example, in this case that's `color=alt.condition(...)`.

### Point selection binding
While selection parameters may be configured statically as in the examples above, they are most powerful when bound to other components of the chart that may be manipulated interactively. Point selections may be bound to widgets, mark click/hover events, and to legend interactions.

#### Binding point selections to widgets
Point selections may be bound to input widgets. For example, let's extend the previous example to use a dropdown widget to control which country of origin is selected





In [ ]:
# construct dropdown (select) widget
input_dropdown = alt.binding_select(
    options=['Europe', 'Japan', 'USA'],
    name='Region '
)

selection = alt.selection_point(
    fields=['Origin'],
    value="Japan",
    bind=input_dropdown
)

alt.Chart(cars).mark_point().encode(
    alt.X('Horsepower'),
    alt.Y('Miles_per_Gallon'),
    order=alt.condition(selection, alt.value(1), alt.value(0)),
    color=alt.condition(
        selection,
        alt.Color('Origin:N').scale(domain=['Europe', 'Japan', 'USA']),
        alt.value("lightgrey")
    )
).add_params(
    selection
)

Here's an example of binding a point selection parameter to a radio button widget, and using the selection to filter the input data to the scatter plot.


In [ ]:
options = ['Europe', 'Japan', 'USA']
labels = [option + ' ' for option in options]

input_radio = alt.binding_radio(
    options=options + [None],
    labels=labels + ['All'],
    name='Region: '
)
selection = alt.selection_point(
    fields=['Origin'],
    value=None,
    bind=input_radio,
)

alt.Chart(cars).mark_circle(size=60).encode(
    alt.X('Horsepower:Q'),
    alt.Y('Miles_per_Gallon:Q'),
    alt.Color('Origin:N').scale(domain=options),
).add_params(
    selection
).transform_filter(
    selection
)

#### Binding point selections to click/hover
A point selection may also be bound to click or hover interactions on a mark. Here is an example that builds a point selection and uses it to control the size and color of the point that is clicked on. In addition, the `order` encoding channel is used to raise the selected point above the unselected points.

In [ ]:
selection = alt.selection_point()

alt.Chart(cars).mark_circle(size=60).encode(
    alt.X('Horsepower'),
    alt.Y('Miles_per_Gallon'),
    size=alt.condition(selection, alt.value(300), alt.value(60)),
    order=alt.condition(selection, alt.value(1), alt.value(0)),
    color=alt.condition(
        selection,
        alt.Color('Origin:N').scale(domain=['Europe', 'Japan', 'USA']),
        alt.value("lightgrey")
    )
).add_params(
    selection
)

Here are some additional options you can use with `alt.selection_point()`

```python
selection = alt.selection_point(
    on="pointerover",  # Hover instead of click
    empty=False,       # Start with all rows unselected instead of selected
    nearest=True,      # Activate nearest point to the cursor
)
```

See [Vega-Altair Selection Documentation](https://altair-viz.github.io/user_guide/generated/api/altair.selection_point.html) for the full list of configuration options.

#### Fields and Encodings Arguments
When creating point selections using `alt.selection_point`, the `fields` and `encodings` optional arguments may be used to specify how selections are made and what data they correspond to.


##### Fields Argument
The `fields` argument specifies which data columns will be used to build the selection predicate. This means that when a selection is made, only the data points that match the specified fields of the selected point will be considered selected.

To see how this works we'll set `fields` to `Origin`, which will cause all points that match the origin of the clicked point to be selected.

##### Encodings Argument
As an alternative to specifying the data fields to consider in the selection, the `encodings` argument allows you to specify the visual encodings that should be used for the selection. This is useful for creating interactions based on how data is represented visually, such as by color, size, or position.

Here's an example of selecting cars that match the color and size of the clicked point.


In [ ]:
selection = alt.selection_point(encodings=["size", "color"])

alt.Chart(cars).mark_circle(size=60).encode(
    alt.X('Horsepower'),
    alt.Y('Miles_per_Gallon'),
    alt.Size('Cylinders:O'),
    order=alt.condition(selection, alt.value(1), alt.value(0)),
    color=alt.condition(
        selection,
        alt.Color('Origin:N').scale(domain=['Europe', 'Japan', 'USA']),
        alt.value("lightgrey")
    )
).add_params(
    selection
)

#### Binding point selections to legend interactions
Point selection parameters may also be bound to legends, which then allows users to drive selections by clicking on legend entries. This is a powerful way to highlight or filter data points based on categorical variables represented in the legend.

A selection is bound to a legend by setting the `bind` argument to the string `'legend'`, and setting the `encodings` argument to a single element list containing the legend's encoding (e.g. `color`, `size`, etc.).

In [ ]:
# Create a point selection bound to legend
selection = alt.selection_point(
    encodings=['color'],
    bind='legend'
)

# Create chart with point selection bound to legend
chart = alt.Chart(cars).mark_circle(size=100).encode(
    alt.X('Horsepower'),
    alt.Y('Miles_per_Gallon'),
    order=alt.condition(selection, alt.value(1), alt.value(0)),
    color=alt.condition(selection, 'Origin:N', alt.value('lightgray'))
).add_params(
    selection
)

chart

### Interval selections
Interval selection parameters are created using `alt.selection_interval()`, and they represent predicates of the form `{lower} <= {Column} < {upper}`. When an interval selection parameter is added to a chart with `add_params`, Vega-Altair adds a rectangle mark to show the selected region. By default, this rectangle mark is draggable and may be resized using scroll zoom.

Here is an example that uses an interval selection with a `condition` to control the mark's color encoding.

In [ ]:
# Define interval selection with initial values
selection = alt.selection_interval(value={'x': [50, 150], 'y': [10, 30]})

# Create chart with point selection bound to legend
chart = alt.Chart(cars).mark_circle(size=100).encode(
    alt.X('Horsepower'),
    alt.Y('Miles_per_Gallon'),
    color=alt.condition(selection, 'Origin:N', alt.value('lightgray'))
).add_params(
    selection
)

chart

The initial interval selection `value` is not required. If not provided, nothing will be selected in the initial chart. Additionally, the `encodings` argument may be used to customize which dimensions may be selected. For example, setting `encodings` to `["x"]` will result in a horizontal box selection

In [ ]:
# Define interval selection with initial values
selection = alt.selection_interval(encodings=["x"])

# Create chart with point selection bound to legend
chart = alt.Chart(cars).mark_circle(size=100).encode(
    alt.X('Horsepower'),
    alt.Y('Miles_per_Gallon'),
    color=alt.condition(selection, 'Origin:N', alt.value('lightgray'))
).add_params(
    selection
)

chart

### Multi-view selections
Selections are particularly useful in multi-view charts, where a selection on one view may be used to filter or highlight marks in other views.

Here we concatenate a new scatter plot, of Acceleration vs Miles per Gallon, to the right of the scatter plot above. Because the two charts are concatenated, we can use the same selection parameter in an `alt.condition` to control the color of the new scatter plot as well.

In [ ]:
selection = alt.selection_interval(encodings=['x'])

left_chart = alt.Chart(cars).mark_circle(size=100).encode(
    alt.X('Horsepower'),
    alt.Y('Miles_per_Gallon'),
    color=alt.condition(selection, 'Origin', alt.value('lightgray'))
).add_params(
    selection
)

right_chart = alt.Chart(cars).mark_circle(size=100).encode(
    alt.X('Acceleration'),
    alt.Y('Miles_per_Gallon'),
    color=alt.condition(selection, 'Origin', alt.value('lightgray'))
)

left_chart | right_chart

You may notice that currently it's only possible to make interval selections on the left chart. This is because we've only added the selection parameter to the left chart using `add_params`. The chart that the selection is added to with `add_params` is the chart that will be configured to drive the selection. As this example demonstrates, a selection may be used in `alt.condition` (or in `transform_filter()`) in any chart in the layout whether or not the selection has been added to the chart with `add_params`.

We can update this example to support selection in both subplot by adding the selection parameter with `add_params` to the right chart as well.

In [ ]:
selection = alt.selection_interval(encodings=['x'])

left_chart = alt.Chart(cars).mark_circle(size=100).encode(
    alt.X('Horsepower'),
    alt.Y('Miles_per_Gallon'),
    color=alt.condition(selection, 'Origin', alt.value('lightgray'))
).add_params(
    selection
)

right_chart = alt.Chart(cars).mark_circle(size=100).encode(
    alt.X('Acceleration'),
    alt.Y('Miles_per_Gallon'),
    color=alt.condition(selection, 'Origin', alt.value('lightgray'))
).add_params(
    selection
)

left_chart | right_chart

Interval selections may also be bound to scales, in which case pan and zoom are activated (rather than box selection) and the selection parameter's predicate is driven by the chart's active viewport.

Here's an example that places two identical scatter plots side by side. A selection parameter with `bind="scales"` is added to the left chart, which
 activates pan and zoom support on this chart. The selection parameter is used inside an `alt.condition` in the right chart to control the color of points. As the user pans and zooms using the left chart, the right chart highlights the points that are in the left chart's viewport.

In [ ]:
selection = alt.selection_interval(bind="scales")

left_chart = alt.Chart(cars).mark_circle(size=100).encode(
    alt.X('Horsepower'),
    alt.Y('Miles_per_Gallon'),
    alt.Color('Origin'),
).add_params(
    selection
)

right_chart = alt.Chart(cars).mark_circle(size=100).encode(
    alt.X('Horsepower'),
    alt.Y('Miles_per_Gallon'),
    color=alt.condition(selection, 'Origin', alt.value('lightgray'))
)

(left_chart | right_chart).properties(bounds="flush", spacing=60)

> **Note:** The `.interactive()` method described earlier is actually just a shortcut for creating an interval selection parameter and adding it to the chart!

In [ ]:
selection = alt.selection_interval()

scatter = alt.Chart(cars).mark_circle(size=100).encode(
    alt.X('Horsepower'),
    alt.Y('Miles_per_Gallon'),
    color=alt.condition(selection, 'Origin', alt.value('lightgray'))
).add_params(
    selection
)

bars = alt.Chart(cars).mark_bar().encode(
    alt.X('count(Origin)').scale(domain=[0,260]),
    alt.Y('Origin').scale(domain=["Europe", "Japan", "USA"]),
    alt.Color('Origin'),
).transform_filter(
    selection
)

scatter & bars

## Jupyter Widget Integration
A powerful feature of Vega-Altair's support for interactivity is that it is implemented entirely in JavaScript, so these interactions work without a Python kernel (e.g. in the Vega Editor).  Sometimes, however, it's really useful to have access to a chart's interactive state in Python in order to use selections to drive other Python logic.

This is possible using [`JupyterChart`](https://altair-viz.github.io/user_guide/jupyter_chart.html), which is built on top of [Jupyter Widgets](https://ipywidgets.readthedocs.io/en/latest/) using [AnyWidget](https://anywidget.dev/).


Here is an example of accessing an interval selection from Python. Note that it's important to name the selection so that it's easy to look up in the `jchart.selections` object.

In [ ]:
brush = alt.selection_interval(name="brush")

chart = alt.Chart(cars).mark_point().encode(
    x='Horsepower:Q',
    y='Miles_per_Gallon:Q',
    color=alt.condition(brush, 'Cylinders:O', alt.value('grey')),
).add_params(brush)

jchart = alt.JupyterChart(chart)
jchart

In [ ]:
jchart.selections.brush.value


Here's a more complex example that uses the selection to filter a pandas DataFrame, then display that filtered pandas DataFrame below that chart in a separate widget.


In [ ]:
from ipywidgets import HTML, VBox

cars = data.cars()
brush = alt.selection_interval(name="brush")

chart_widget = alt.JupyterChart(alt.Chart(cars).mark_point().encode(
    x='Horsepower:Q',
    y='Miles_per_Gallon:Q',
    color=alt.condition(brush, 'Cylinders:O', alt.value('grey')),
).add_params(brush))

table_widget = HTML(value=cars.iloc[:0].to_html())

def on_select(change):
    sel = change.new.value
    if sel is None or 'Horsepower' not in sel:
        filtered = cars.iloc[:0]
    else:
        filter_query = (
            f"{sel['Horsepower'][0]} <= `Horsepower` <= {sel['Horsepower'][1]} and "
            f"{sel['Miles_per_Gallon'][0]} <= `Miles_per_Gallon` <= {sel['Miles_per_Gallon'][1]}"
        )
        filtered = cars.query(filter_query)

    table_widget.value = filtered.to_html()

chart_widget.selections.observe(on_select, ["brush"])

VBox([chart_widget, table_widget])